A notebook to test Pytorch.

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import numpy as np
import os
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import matplotlib
import matplotlib.pyplot as plt

# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 24
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [7]:
torch.zeros(3, device=torch.device('cpu'))

tensor([0., 0., 0.])

In [11]:
w = torch.tensor([3,5,4.0])
w.dot(torch.ones(3))

tensor(12.)

## Torch 2d to 1d and vice versa

In [223]:
torch.eye(3).matmul(torch.tensor([[2., 3, 1]]).T)

tensor([[2.],
        [3.],
        [1.]])

In [222]:
torch.tensor([[2., 3, 1]]).T

tensor([[2.],
        [3.],
        [1.]])

In [30]:
V = torch.randn(3, 1)
V

tensor([[0.9760],
        [0.6724],
        [1.2658]])

In [31]:
V.reshape(-1)

tensor([0.9760, 0.6724, 1.2658])

In [34]:
v = torch.rand(4)
v

tensor([0.5324, 0.6466, 0.7210, 0.6114])

In [219]:
v - 1

tensor([-0.4676, -0.3534, -0.2790, -0.3886])

In [35]:
v.reshape(4, 1)

tensor([[0.5324],
        [0.6466],
        [0.7210],
        [0.6114]])

## Torch distributions
https://pytorch.org/docs/stable/distributions.html

In [12]:
import torch.distributions as dists

In [24]:
dis = dists.Normal(0, 1)
dis.log_prob(torch.ones(5))

tensor([-1.4189, -1.4189, -1.4189, -1.4189, -1.4189])

### Grad of log (Normal density)

$$ \nabla_x \log p(x) = -x$$ where $p(x) = \mathcal{N}(0,1)$.

Input is a scalar

In [109]:
x = torch.tensor([3.0], requires_grad=True)
y = 2*x
lp = dis.log_prob(y)

In [102]:
lp.backward(retain_graph=True) 
x.grad

tensor([-12.])

In [104]:
torch.autograd.grad([lp], [x, y], retain_graph=True, only_inputs=True)

(tensor([-12.]), tensor([-6.]))

Input is n x 1

In [186]:
X = torch.tensor([[-3, 1.0, 2]], requires_grad=True).T
Lp = dis.log_prob(X)
sum_lp = torch.sum(Lp)

In [187]:
torch.autograd.grad(sum_lp, X, retain_graph=True, only_inputs=True)

(tensor([[ 3.],
         [-1.],
         [-2.]]),)

## Multivariate normal

Assume $p(x) = \mathcal{N}(x \mid \mu, I)$. Then

$$ \nabla_x \log p(x) = -(x-\mu) $$

In [212]:
mdist = dists.MultivariateNormal(
    loc=torch.tensor([1, 2]), covariance_matrix=torch.eye(2))

X = torch.tensor([
    [0, 0.0],
    [0, 1.0],
    [1.0, 4],
])
X.requires_grad = True
logp = mdist.log_prob(X)
torch.autograd.grad(torch.sum(logp), X, retain_graph=True, only_inputs=True)[0]

tensor([[ 1.,  2.],
        [ 1.,  1.],
        [ 0., -2.]])

## Broadcasting

In [230]:
a = torch.arange(1, 4).reshape(3, 1)
a

tensor([[1],
        [2],
        [3]])

In [232]:
b = torch.arange(3, 7).reshape(1, 4)
b

tensor([[3, 4, 5, 6]])

In [234]:
a+b

tensor([[4, 5, 6, 7],
        [5, 6, 7, 8],
        [6, 7, 8, 9]])

In [237]:
a + b.reshape(-1)

tensor([[4, 5, 6, 7],
        [5, 6, 7, 8],
        [6, 7, 8, 9]])

In [238]:
torch.exp

<function _VariableFunctions.exp>

In [240]:
torch.zeros((3,4))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

## Remove diagonal

In [241]:
A = torch.arange(16).reshape(4,4)
A

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [249]:
diagA = torch.diagflat(torch.diag(A))
diagA

tensor([[ 0,  0,  0,  0],
        [ 0,  5,  0,  0],
        [ 0,  0, 10,  0],
        [ 0,  0,  0, 15]])

In [250]:
A - diagA

tensor([[ 0,  1,  2,  3],
        [ 4,  0,  6,  7],
        [ 8,  9,  0, 11],
        [12, 13, 14,  0]])

## Multinomial

In [264]:
d = 5
m = 20
mult = dists.multinomial.Multinomial(total_count=d, probs=torch.ones(d)/d)
mult.sample()

tensor([2., 0., 1., 0., 2.])

## Quadratic form
   

In [275]:
w = torch.rand(5)
A = torch.randn(5,5)

In [276]:
A.matmul(w).matmul(w)

tensor(-0.4280)

## Bool to float

In [277]:
v = torch.randn(5)
v

tensor([-0.4104, -0.4368,  0.2770, -0.0769, -2.0267])

In [279]:
I = v >  0
I

tensor([False, False,  True, False, False])

In [283]:
I.to(torch.float)

tensor([0., 0., 1., 0., 0.])

## Torch scalar to normal float

In [290]:
torch_num = torch.tensor(3.78)
torch_num.item()

3.7799999713897705

## Elementwise product

In [295]:
A = torch.randn(5,2)
B = torch.rand(5, 2)
torch.sum(A*B, 1)

tensor([ 1.0602, -0.1103, -0.5013, -0.1123,  0.0481])

## Meshgrid


In [306]:
a = torch.arange(5.0)
print(a)
b = torch.rand(4)
print(b)

tensor([0., 1., 2., 3., 4.])
tensor([0.5099, 0.2987, 0.6258, 0.9401])


In [305]:
torch.meshgrid(a,b)

RuntimeError: Expected scalar or 1D tensor in the tensor list but got:  0.4967
 0.8643
 0.6685
 0.7017
[ torch.FloatTensor{4,1} ]